In [3]:
from langchain_openai import ChatOpenAI
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import *
from langchain_core.output_parsers import *
from langchain.chat_models import init_chat_model
from langchain_core.runnables import *
from langchain_core.messages import *

In [4]:
#model = init_chat_model("gpt-4o-mini", model_provider="openai")
#model = init_chat_model("open-mistral-nemo", model_provider="mistralai")
#OR 
model = ChatMistralAI(
    model="open-mistral-nemo",
    temperature=0,
    max_retries=2,
    # other params...
)


messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='"Ciao!"', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 13, 'total_tokens': 18, 'completion_tokens': 5}, 'model': 'open-mistral-nemo', 'finish_reason': 'stop'}, id='run-01479f4e-0225-4de9-ace4-d95eb34dc140-0', usage_metadata={'input_tokens': 13, 'output_tokens': 5, 'total_tokens': 18})

In [9]:
## Usage of prompts 
st = "Translate from English into {language}"
pt = ChatPromptTemplate.from_messages([("system", st),("user", "{text}")])
#Almost everything in Langchain is derived from Runnable 
prompt = pt.invoke({'language': 'Italian', 'text': 'hi!'})
prompt.to_messages()
model.invoke(prompt)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 11, 'total_tokens': 15, 'completion_tokens': 4}, 'model': 'open-mistral-nemo', 'finish_reason': 'stop'}, id='run-4fde70c1-f945-4944-a9c3-bbd319883215-0', usage_metadata={'input_tokens': 11, 'output_tokens': 4, 'total_tokens': 15})

In [12]:
#What is chains
#There are various kind of output parsers
prompt = PromptTemplate.from_template("Describe the {topic} in 3 sentences")
chain = prompt | model | StrOutputParser()
chain.batch([{'topic': 'LLM'}, {'topic': 'India'}])

["The Large Language Model (LLM) is a type of artificial intelligence model designed to understand, generate, and interact with human language. It's trained on vast amounts of text data to predict the next word in a sentence, enabling it to generate coherent and contextually relevant text. LLMs can perform a wide range of tasks, such as answering questions, translating languages, and even creating creative content like poetry or stories.",
 "India, the world's second most populous country, is known for its diverse culture, vibrant festivals like Diwali and Holi, and a rich history spanning over 5,000 years, including iconic landmarks like the Taj Mahal. It's also renowned for its vast array of cuisines, from street food like chaat to complex dishes like biryani, and its spiritual traditions, including Hinduism, Buddhism, and yoga."]

In [17]:
#Do async abatch_as_completed
#must be inside async def 
#and since it gives AsyncIterator
#async for 
#then run it using asyncio.run()
async def get_output():
    it = chain.abatch_as_completed([{'topic': 'LLM'}, {'topic': 'India'}])
    async for e in it:
        print(e)
#jupyter
await  get_output()
#asyncio.run(get_output())


(0, "The Large Language Model (LLM) is a type of artificial intelligence model designed to understand, generate, and interact with human language. It's trained on vast amounts of text data to predict the next word in a sentence, enabling it to generate coherent and contextually relevant text. LLMs can perform a wide range of tasks, such as answering questions, translating languages, and even creating creative content like poetry or stories.")
(1, "India, the world's second most populous country, is known for its diverse culture, vibrant festivals like Diwali and Holi, and a rich history spanning over 5,000 years, including iconic landmarks like the Taj Mahal. It's also renowned for its vast array of cuisines, from street food like chaat to complex dishes like biryani, and its spiritual traditions, including Hinduism, Buddhism, and yoga.")


In [ ]:
#complex Prompt
template = """
You are a seasoned English teacher with 10 years of experience. 
Please write an English conversation suitable for the given situation.  
Refer to the [FORMAT] for the structure.

#SITUATION:
{question}

#FORMAT:
- Dialogue in English:
- Explanation of the Dialogue: 
"""
#create a chain with str output parser and invoke with below 
{"question": "I want to go to a restaurant and order food."}


In [ ]:
# other components of chain 
#Parallel
chain1 = ( PromptTemplate.from_template("What is the capital of {country}? Give it in one word")
          | model 
          | StrOutputParser()
         )
chain2 = ( PromptTemplate.from_template("What is the area of {country}? Give it in one number")
          | model
          | StrOutputParser()
         )
cchain = RunnableParallel(capital=chain1, area=chain2)
cchain.invoke({'country': 'India'})

In [24]:
# RunnablePassthrough, RunnableLambda , RunnableParallel
rn = RunnableParallel(original= RunnablePassthrough(),
                      modified= lambda x: x
) #dict of output where key is keyword arg of its calling 
#Passthrough gets back what u provide in invoke
rn.invoke({'x': 2})  #x 

{'original': {'x': 2}, 'modified': {'x': 2}}

In [26]:
#For example 
def fake_llm(prompt: str)->str:
    return f"{len(prompt)}-{prompt}"
#RunnableLambda converts a py function into Runnable 
chain = RunnableLambda(fake_llm) | RunnableParallel(
    original = RunnablePassthrough(),
    modified = lambda text: text[0]
)
chain.invoke("OK")  #? {'original': 2-OK, 'modified': 2}

{'original': '2-OK', 'modified': '2'}

In [30]:
#REal example 
import datetime 
def get_today(arg=None):
    return datetime.date.today().strftime('%b-%d')
get_today()

prompt = PromptTemplate.from_template("""List {n} famous people whose birthday is on {today} 
and also list their actual birth date """)

#dict is coerced into RunnableParallel
chain = (
    { 'today': RunnableLambda(get_today), 'n' : RunnablePassthrough()} 
    | prompt
    | model 
    | StrOutputParser()
)
chain.invoke(3)

'Here are three famous people who were born on May 2nd, along with their birth years:\n\n1. **Marilyn Monroe** - Born on May 2, 1926, in Los Angeles, California, USA. She was an American actress, model, and singer who became a major sex symbol, starring in a number of successful films during the 1950s.\n\n2. **Heath Ledger** - Born on May 2, 1979, in Perth, Australia. He was an Australian actor and director. Ledger was the recipient of numerous accolades, including the Academy Award for Best Supporting Actor, a Golden Globe Award, and four Australian Film Institute Awards.\n\n3. **David Beckham** - Born on May 2, 1975, in Leytonstone, London, England. He is an English former professional footballer, the current president of Inter Miami CF and co-owner of Salford City. He played for Manchester United, Preston North End, Real Madrid, Milan, LA Galaxy, and the England national team.'

In [32]:
##Agents using langchain, Nowadays people use langgraph
from langchain_core.tools import tool
from langchain_mistralai import ChatMistralAI
from langchain_core.messages import *
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import * 
from langchain_core.agents import * 
from langchain.agents.tool_calling_agent.base import create_tool_calling_agent
from  langchain.agents.agent import AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import time 

In [38]:
#simple AGents 
#must have types
#must have doc string 
#based on above, LLM selects the function
@tool
def add(a:int, b:int)->int:
    "adds two numbers a and b"
    return a+b 

@tool
def multiply(a:int, b:int) -> int:
    "multiply two numbers a and b"
    return a*b
    

In [39]:
tools = [add, multiply]
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ('human', '{query}'),
    #below is must, where agen will do temp work
    ('placeholder', '{agent_scratchpad}')
])
agent = create_tool_calling_agent(model, tools, prompt)
query = "what is 3+1? what is 4*5?"


In [41]:
aexec = AgentExecutor(agent=agent, tools=tools)
aexec.invoke(dict(query=query))

{'query': 'what is 3+1? what is 4*5?', 'output': '3+1=4, 4*5=20'}

In [48]:
#adds memory
prompt = prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    #Exactly at this place as it should go before user query 
    ('placeholder', '{chat_history}'),
    ('human', '{query}'),
    #below is must, where agen will do temp work
    ('placeholder', '{agent_scratchpad}')
])

history = {}
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in history:
            history[session_id] = ChatMessageHistory()
    return history[session_id]
    
tools = [add, multiply]
agent = create_tool_calling_agent(model, tools, prompt)
aexec = AgentExecutor(agent=agent, tools=tools)
agent_with_mem = RunnableWithMessageHistory(
    aexec,
    get_session_history,
    input_messages_key = "query",
    history_messages_key = 'chat_history'
)


In [49]:
queries = [
    "what is 3+1? what is 4*5?",
    "I am Das",
    "Who am I? Tell me the result again",
]
conf = {'configurable': {'session_id': 'my-session'}}
for q in queries:
    msg = dict(query=q)
    time.sleep(2)
    resp = agent_with_mem.invoke(msg, conf)
    print(resp['output'])

3+1=4, 4*5=20
Hello Das, how can I help you?
You are Das. Here are the results again:

* 3+1=4
* 4*5=20


In [50]:
history

{'my-session': InMemoryChatMessageHistory(messages=[HumanMessage(content='what is 3+1? what is 4*5?', additional_kwargs={}, response_metadata={}), AIMessage(content='3+1=4, 4*5=20', additional_kwargs={}, response_metadata={}), HumanMessage(content='I am Das', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Das, how can I help you?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Who am I? Tell me the result again', additional_kwargs={}, response_metadata={}), AIMessage(content='You are Das. Here are the results again:\n\n* 3+1=4\n* 4*5=20', additional_kwargs={}, response_metadata={})])}

In [51]:
##RAG 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings  


In [53]:
# Step 1: Load Documents
loader = PyMuPDFLoader("A European Approach to Artificial Intelligence - A Policy Perspective.pdf")
docs = loader.load()
print(f"Number of pages in the document: {len(docs)}")

Number of pages in the document: 24


In [60]:
#Step2,3,4
# Step 2: Split Documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"Number of split chunks: {len(split_documents)}")
#create embeddings 
from langchain_mistralai import MistralAIEmbeddings
embeddings = MistralAIEmbeddings(model="mistral-embed",)
# Create a vector store.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)


Number of split chunks: 163


In [61]:
#Create a retriever , 
#create a chain and then invoke 
retriever = vectorstore.as_retriever()
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

#Context: 
{context}

#Question:
{question}

#Answer:"""
)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
question = "Where has the application of AI in healthcare been confined to so far?"
response = chain.invoke(question)
print(response)

Based on the provided context, the application of AI in healthcare has been confined to two main domains so far:

1. **Administrative Tasks**: This includes using Natural Language Processing to extract information from clinical notes or predictive scheduling of visits.

2. **Diagnostic Tasks**: This involves the use of machine and deep learning in imaging, radiology, pathology, and dermatology. These applications have led to reduced time for healthcare professionals on routine tasks and more accurate and faster diagnostics.
